In [1]:
import tensorflow as tf
from tensorflow import keras
import numpy as np
import os

In [2]:
filepath = 'datasets/11/capek/valka_s_mloky.txt'
with open(filepath) as f:
    capek_text = f.read()

In [4]:
print(capek_text[:200])

Karel Čapek
VÁLKA S MLOKY


Znění tohoto textu vychází z díla Válka s mloky tak, jak bylo vydáno v Československém spisovateli v roce 1981 (ČAPEK, Karel. Válka s mloky.  20. vyd. Praha : Československ


**Převod na tokeny**

In [5]:
"".join(sorted(set(capek_text.lower())))

'\t\n !%&()*,-./0123456789:;?[]abcdefghijklmnopqrstuvwxyz°áâçéíîóôöúüýčďęěňŕřšťůűž–‘’“”…'

In [6]:
tokenizer = keras.preprocessing.text.Tokenizer(char_level=True)
tokenizer.fit_on_texts(capek_text)

In [8]:
max_id = len(tokenizer.word_index) # pocet ruznych znaku
dataset_size = tokenizer.document_count # celkovy pocet znaku

In [9]:
max_id

85

**Převod celého korpusu na tokeny**

In [10]:
[encoded] = np.array(tokenizer.texts_to_sequences([capek_text])) - 1
train_size = dataset_size * 90 // 100 # pro trenovani pouzijeme 90 % datasetu
dataset = tf.data.Dataset.from_tensor_slices(encoded[:train_size]) # sekvence znak po znaku

In [11]:
encoded.shape

(410451,)

**Příprava datasetu pomocí knihovny tf.data**

window() nám převede dlouhou sekvenci (1M znaků) na okna o délce X (zde 100) znaků. shift=1 zajistí, že první instance jsou znaky 0-100, druhá instance 1-101 atd.

In [12]:
n_steps = 100
window_length = n_steps + 1 # target = input posunutý o 1 znak dopředu
dataset = dataset.repeat().window(window_length, shift=1, drop_remainder=True)

window() nám dataset převede na dataset datasetů, což nemůžeme použít pro trénování. proto to splácneme zpátky na dataset sekvencí

In [13]:
dataset = dataset.flat_map(lambda window: window.batch(window_length))

GD funguje nejlépe, když jsou trénovací instance náhodně distribuované, proto dataset zamícháme a připravíme do dávek. Zároveň sekvenci o 101 znacích převedeme na prvních 100 znaků trénovacích a posledních 100 znaků jako target 

In [14]:
batch_size = 32
dataset = dataset.shuffle(10000).batch(batch_size)
dataset = dataset.map(lambda windows: (windows[:, :-1], windows[:, 1:]))

Protože různých znaků je málo, vystačíme si s one-hot encoding namísto word embeddings. Targety můžeme nechat jako celočíselné indexy. 

In [15]:
dataset = dataset.map(
    lambda X_batch, y_batch: (tf.one_hot(X_batch, depth=max_id), y_batch))

Prefetching předchystává dávky dat, aby byly nachystané už ve chvíli, kdy si je model vyžádá a nečekalo se na ně

In [16]:
dataset = dataset.prefetch(1)

**Zkontrolujeme výstup datasetu**

In [17]:
for X_batch, y_batch in dataset.take(1):
    print(X_batch.shape, y_batch.shape)

(32, 100, 85) (32, 100)


In [18]:
X_batch[0]

<tf.Tensor: shape=(100, 85), dtype=float32, numpy=
array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]], dtype=float32)>

In [19]:
y_batch[0]

<tf.Tensor: shape=(100,), dtype=int32, numpy=
array([18,  6,  2,  0,  9, 32,  8,  9, 29,  0, 23,  3,  5,  3, 12, 19,  0,
       43, 22,  0, 20,  2,  0,  1, 14,  8,  6, 31,  0,  1, 23, 16, 18,  1,
       11,  4, 17,  0,  3, 37,  2,  4,  5, 22,  0, 12, 27, 17, 28,  2,  4,
        2, 16,  0, 10,  2, 25,  8,  0, 12, 15, 23, 15,  0,  3,  0, 14,  1,
       13,  5, 15, 37,  3,  6, 16,  2, 10,  0,  3,  0, 20,  2, 32,  5, 24,
        0,  9, 24,  5, 32, 17,  0, 25,  6,  1, 11, 24, 20, 22,  0])>

**Připravíme a natrénujeme model**

Pokud nám má Dense vrstva vracet sekvenci, tak ji obalíme do TimeDistributed vrstvy, která se o to postará.

In [23]:
model = keras.models.Sequential([
    keras.layers.GRU(128, return_sequences=True, input_shape=[None, max_id],
                     dropout=0.2),
    keras.layers.TimeDistributed(keras.layers.Dense(max_id,
                                                    activation="softmax"))
])
model.compile(loss="sparse_categorical_crossentropy", optimizer="adam")

In [24]:
model.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 gru_2 (GRU)                 (None, None, 128)         82560     
                                                                 
 time_distributed_1 (TimeDis  (None, None, 85)         10965     
 tributed)                                                       
                                                                 
Total params: 93,525
Trainable params: 93,525
Non-trainable params: 0
_________________________________________________________________


In [25]:
history = model.fit(dataset, steps_per_epoch=train_size // batch_size,
                    epochs=5)

Epoch 1/5
  959/11543 [=>............................] - ETA: 12:43 - loss: 2.6698

KeyboardInterrupt: 

In [26]:
model.save('11_shakespeare.keras')

In [86]:
model = keras.models.load_model('11_shakespeare.keras')

**Generování znaků a sekvencí z natrénovaného modelu**

Protože jsme si upravovali ID tokenů, musíme si na to nachystat preprocessing i pro predikce

In [87]:
def preprocess(texts):
    X = np.array(tokenizer.texts_to_sequences(texts)) - 1
    return tf.one_hot(X, max_id)

Protože jsme model učili predikovat sekvenci, vrátí nám metoda predict zase sekvenci. Znak, který by měl následovat po vstupních datech, je posledním znakem. 

In [88]:
X_new = preprocess(["How are yo"])
X_new

<tf.Tensor: shape=(1, 10, 39), dtype=float32, numpy=
array([[[0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0.],
        [1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0.

In [89]:
y_pred = np.argmax(model.predict(X_new), axis=-1)
y_pred

array([[ 1, 13,  0,  2,  0,  1,  0,  2,  3, 13]], dtype=int64)

In [90]:
tokenizer.sequences_to_texts(y_pred + 1)[0][-1] # ID zase o 1 zvedneme 

'u'

In [91]:
tokenizer.sequences_to_texts(y_pred + 1)[0]

'e u   t   e   t o u'

Pomocí temperature můžeme stanovit, jak moc náhodná má být volba dalšího znaku z možných predikovaných

In [92]:
def next_char(text, temperature=1):
    X_new = preprocess([text])
    y_proba = model.predict(X_new)[0, -1:, :]
    rescaled_logits = tf.math.log(y_proba) / temperature
    char_id = tf.random.categorical(rescaled_logits, num_samples=1) + 1
    return tokenizer.sequences_to_texts(char_id.numpy())[0]

In [93]:
next_char("How are yo", temperature=1)

'u'

In [94]:
def complete_text(text, n_chars=50, temperature=1):
    for _ in range(n_chars):
        text += next_char(text, temperature)
    return text

In [98]:
print(complete_text("t", temperature=0.2))

the sense of the stole of the store--

first citize


In [99]:
print(complete_text("t", temperature=1))

t
to whom sweets knock her home

isabella:
and sinc


In [100]:
print(complete_text("t", temperature=2))

t objoh'dous, jock..'mansduous planieat; claunetl r
